#CHAPTER 11

In [ ]:
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Conv1D, GlobalMaxPooling1D, LSTM, GRU, Bidirectional, Dense, Flatten


In [ ]:
# Load the dataset
dataset, info = tfds.load('ag_news_subset', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [ ]:
# Tokenize and pad the sequences
tokenizer = Tokenizer(num_words=20000, oov_token="<OOV>")
train_texts = [x[0].numpy().decode('utf-8') for x in train_dataset]
tokenizer.fit_on_texts(train_texts)
sequences = tokenizer.texts_to_sequences(train_texts)
sequences = pad_sequences(sequences, padding='post')

In [ ]:
# Convert labels to one-hot encoding
train_labels = [label.numpy() for _, label in train_dataset]
train_labels = to_categorical(train_labels, num_classes=4)  # assuming 4 classes

# Split the training set into training and validation sets
train_sequences, val_sequences, train_labels, val_labels = train_test_split(sequences, train_labels, test_size=0.2, random_state=42)


In [ ]:
vocab_size=20000
embedding_dim =64
max_length=sequences.shape[1]

In [ ]:
# Define the DNN model
model_dnn = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    Dense(4, activation='softmax')
])

# Define the CNN model
model_cnn = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    Dense(4, activation='softmax')
])

# Define the LSTM model
model_lstm = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(32, return_sequences=True),
    LSTM(32),
    tf.keras.layers.Dense(64, activation='relu'),
    Dense(4, activation='softmax')
])


# Define the BiLSTM model
model_BiLSTM = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(32, return_sequences=True)),
    Bidirectional(LSTM(16)),
    tf.keras.layers.Dense(64, activation='relu'),  # new layer
    Dense(4, activation='softmax')
])


models = [model_cnn, model_dnn, model_lstm, model_BiLSTM]

for model in models:
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(train_sequences, train_labels, epochs=10, validation_data=(val_sequences, val_labels),verbose=False)



In [ ]:
# Evaluate the model
test_texts = [x[0].numpy().decode('utf-8') for x in test_dataset]
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_sequences = pad_sequences(test_sequences, padding='post', maxlen=sequences.shape[1])

test_labels = [label.numpy() for _, label in test_dataset]
test_labels = to_categorical(test_labels, num_classes=4)

model_names = ["Model_CNN", "Model_DNN", "Model_LSTM", "Model_BiLSTM"]

for i, model in enumerate(models):
  loss, accuracy = model.evaluate(test_sequences, test_labels)

  print("Model Evaluation -", model_names[i])
  print("Loss:", loss)
  print("Accuracy:", accuracy)
  print()

238/238 [==============================] - 1s 4ms/step - loss: 0.7756 - accuracy: 0.8989
Model Evaluation - Model_CNN
Loss: 0.7755934000015259
Accuracy: 0.8989473581314087

238/238 [==============================] - 1s 2ms/step - loss: 0.7091 - accuracy: 0.8896
Model Evaluation - Model_DNN
Loss: 0.7091193199157715
Accuracy: 0.8896052837371826

238/238 [==============================] - 2s 7ms/step - loss: 0.3211 - accuracy: 0.9008
Model Evaluation - Model_LSTM
Loss: 0.32113003730773926
Accuracy: 0.9007894992828369

238/238 [==============================] - 4s 10ms/step - loss: 0.5618 - accuracy: 0.8916
Model Evaluation - Model_BiLSTM
Loss: 0.5618014335632324
Accuracy: 0.8915789723396301

